In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
import plotly
import math
from statistics import median, mean
from selfpkg import readDB, MinToMS
from dataFilter import butter_lowpass_filter, butter_highpass_filter

In [2]:
timeConversionVal = 60000

In [3]:
#function for calculate the time interval between peaks
def velocityCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = dfTime[i] - prev #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [4]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [129]:
#function for generating compass chart
def compassChart(df, filename='compassChart'):
    
    x = df.compassX
    y = df.compassY
    z = df.compassZ

    v = (x ** 2 + y ** 2 + z ** 2) ** (1/2)
    
    traceCompass = go.Scattergl(
        x=df.time / timeConversionVal,
        y=v,
        name = 'compassSum',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    
#     v_processed = butter_highpass_filter(v, 0.5, 50, 5)
    v_processed = v
    windowSize = 250
    avg = movingaverage(v_processed, windowSize)
    
    threshold = 3.2
    for i in range(len(v_processed)):
        v_processed[i] = v_processed[i] - avg[i]
    
    traceCompassShift = go.Scattergl(
        x = df.time / timeConversionVal,
        y = v,
        name = 'compass shift',
        line = dict(
            color = ('rgb(25, 25, 25)'),
            width = 1
        )
    )
    
    for i in range(len(v_processed)):
        if v_processed[i] < threshold and v_processed[i] > -threshold:
            v_processed[i] = 0
    
    traceProcessedCompass = go.Scattergl(
        x=df.time / timeConversionVal,
        y=v_processed,
        name = 'compassProcessed',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )

    data = [traceCompass, traceCompassShift, traceProcessedCompass]
    
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'compass sensor output'))
                 

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')
    
    return v_processed

In [133]:
run = 3
df = pd.read_csv('/Users/meng/Downloads/Wise_Run' + str(run) + '_Trimed.csv')
# time = 16.5
# time = MinToMS(time, df)
# index = df.index[df['time'] == time].tolist()[0]
# df = df[:index]
v_processed = compassChart(df, filename='compassChart')
copper, iron = metalDetectionByCompass(df, v_processed, filename='metalDetectionByCompass05')
# print(copper)
# print(iron)

In [132]:
def metalDetectionByCompass(df, v, filename='metalDetectionByCompass'):

#     i = 0
#     j = 0
#     width = 25 #ger rid of peak whose width > 25 points
#     while(j < len(v_processed)):
#         if v_processed[i] == 0 and v_processed[j] == 0:
#             i = i + 1
#             j = j + 1
#         elif v_processed[i] != 0 and v_processed[j] != 0:
#             j = j + 1
#         elif v_processed[i] != 0 and v_processed[j] == 0:
#             if j - i + 1 > width:
#                 for k in range(i, j):
#                     v_processed[k] = 0
#             i = j;

    v_processed = v
    #generate peaks index
    peaksIndex = []
    peaksIndexPos, _ = ss.find_peaks(v)
    peaksIndexNeg, _ = ss.find_peaks(-v)
    
    peaksIndexPosProcessed = []
    peaksIndexNegProcessed = []
    for i in range(len(peaksIndexPos)):
        if v[peaksIndexPos[i]] > 0:
            peaksIndexPosProcessed.append(peaksIndexPos[i])
    for i in range(len(peaksIndexNeg)):
        if v[peaksIndexNeg[i]] < 0:
            peaksIndexNegProcessed.append(peaksIndexNeg[i])
    peaksIndexPos = peaksIndexPosProcessed
    peaksIndexNeg = peaksIndexNegProcessed
    
    #merge close peaks
    i = 0
    j = 1
    thresInterval = 200 # the smallest interval between two joints
    res = []
    while(j < len(peaksIndexPos)):
        if  j == len(peaksIndexPos) - 1:
            if v_processed[peaksIndexPos[i]] >= v_processed[peaksIndexPos[j]]:
                res.append(peaksIndexPos[i])
            else:
                res.append(peaksIndexPos[j])
            i = j
            j += 1       
        else:
            if peaksIndexPos[j] - peaksIndexPos[j-1] > thresInterval:
                j -= 1
                if v_processed[peaksIndexPos[i]] >= v_processed[peaksIndexPos[j]]:
                    res.append(peaksIndexPos[i])
                else:
                    res.append(peaksIndexPos[j])
                j += 1
                i = j
                j += 1       
            else:
                j += 1

#     res = res[1:]
    
    i = 0
    j = 0
    resNeg = []
    while(j < len(peaksIndexNeg)):
        if  j == len(peaksIndexNeg) - 1:
            if v_processed[peaksIndexNeg[i]] <= v_processed[peaksIndexNeg[j]]:
                resNeg.append(peaksIndexNeg[i])
            else:
                resNeg.append(peaksIndexNeg[j])
            i = j
            j += 1       
        else:
            if peaksIndexNeg[j] - peaksIndexNeg[j-1] > thresInterval:
                j -= 1
                if v_processed[peaksIndexNeg[i]] <= v_processed[peaksIndexNeg[j]]:
                    resNeg.append(peaksIndexNeg[i])
                else:
                    resNeg.append(peaksIndexNeg[j])
                j += 1
                i = j
                j += 1       
            else:
                j += 1

#     resNeg = resNeg[1:]
    i = 0
    j = 0
    while(i < len(res) and j < len(resNeg)):
        if res[i] > resNeg[j]:
            peaksIndex.append(resNeg[j])
            j += 1
        else:
            peaksIndex.append(res[i])
            i += 1
    while(i < len(res)):
        peaksIndex.append(res[i])
        i += 1
    while(j < len(resNeg)):
        peaksIndex.append(resNeg[j])
        j += 1
    
    thresInterval = 200
    final = []
    i = 0
    while i < len(peaksIndex):
        if i != len(peaksIndex) - 1 and \
           v[peaksIndex[i]] * v[peaksIndex[i+1]] < 0 and \
           peaksIndex[i+1] - peaksIndex[i] < thresInterval:
            if abs(v[peaksIndex[i]]) > abs(v[peaksIndex[i+1]]):
                final.append(peaksIndex[i])
            else:
                final.append(peaksIndex[i+1])
            i += 2
        else:
            final.append(peaksIndex[i])
            i += 1

    trace0 = go.Scattergl(
        x = df.time/timeConversionVal,
        y = v,
        name = 'processed compass',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1
        )
    )
    
    trace1 = go.Scattergl(
        x = df.time/timeConversionVal,
        y = (df.compassX ** 2 + df.compassY ** 2 + df.compassZ ** 2) ** (1/2),
        name = 'compass',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1
        )
    )

#     trace2 = go.Scattergl(
#         x = df.time[res] / timeConversionVal,
#         y = v[res],
#         mode = 'markers',
#         marker = dict(
#             color = ('rgb(225, 0, 0)'),
#             size = 5
#         )
#     )

#     trace3 = go.Scattergl(
#         x = df.time[peaksIndexPos] / timeConversionVal,
#         y = v[peaksIndexPos],
#         mode = 'markers',
#         marker = dict(
#             color = ('rgb(0, 0, 225)'),
#             size = 5
#         )
#     )
    
#     trace4 = go.Scattergl(
#         x = df.time[resNeg] / timeConversionVal,
#         y = v[resNeg],
#         mode = 'markers',
#         marker = dict(
#             color = ('rgb(225, 0, 0)'),
#             size = 5
#         )
#     )

#     trace5 = go.Scattergl(
#         x = df.time[peaksIndexNeg] / timeConversionVal,
#         y = v[peaksIndexNeg],
#         mode = 'markers',
#         marker = dict(
#             color = ('rgb(0, 0, 225)'),
#             size = 5
#         )
#     )
    finalPos = []
    finalNeg = []
    for i in range(len(final)):
        if v[final[i]] > 0:
            finalPos.append(final[i])
        else:
            finalNeg.append(final[i])
    
    traceResPos = go.Scattergl(
        x = df.time[finalPos] / timeConversionVal,
        y = v[finalPos],
        name = 'Copper',
        mode = 'markers',
        marker = dict(
            color = ('rgb(0, 0, 225)'),
            size = 5
        )
    )
    
    traceResNeg = go.Scattergl(
        x = df.time[finalNeg] / timeConversionVal,
        y = v[finalNeg],
        name = 'iron',
        mode = 'markers',
        marker = dict(
            color = ('rgb(225, 0, 0)'),
            size = 5
        )
    )

    data = [trace0, trace1, traceResPos, traceResNeg]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Megnetic')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')
    
    return (df.time[finalPos] / timeConversionVal).tolist(), (df.time[finalNeg] / timeConversionVal).tolist()